In [1]:
import numpy as np
import torch
import copy
from torch.utils.data import Dataset, DataLoader
from torch import nn
import matplotlib.pyplot as plt
import librosa
import librosa.display
from tqdm import tqdm
import json
import os
from sklearn.model_selection import train_test_split

In [2]:
seed = 1
torch.manual_seed(seed)
chord_templates:dict = json.load(open('./chord_templates.json'))

In [3]:
class ChordDetector(Dataset):
    def __init__(self, train:bool, chord_template:dict = json.load(open('./chord_templates.json')), data_location:str = './data/', sr = 44100, hop = 256):
        super(ChordDetector, self).__init__()
        self.chord_template = chord_template
        self.data = []
        self.sr = sr
        self.hop = hop
        for file in os.listdir(data_location):
            chord_true = torch.Tensor(self.chord_template[self._extract_chord_name(file)])
            y, sr = librosa.load(data_location+file, sr = sr)
            chroma = torch.Tensor(librosa.feature.chroma_cens(y=y, sr = sr, hop_length=hop)).T
            self.data.append((chroma, chord_true))
        X_train, X_test, _, _ = train_test_split(self.data, self.data, test_size=0.2, random_state=seed)
        if train:
            self.data = X_train
        else:
            self.data = X_test

    def _extract_chord_name(self, file):
        main = file[:file.index('-')]
        if file[file.index('-')+1]=='D':
            return main+'dim'
        elif file[file.index('-')+1]=='A':
            return main+'aug'
        elif file[file.index('-')+2]=='i':
            return main+'m'
        return main
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

In [4]:
train_data = ChordDetector(train=True)
val_data = ChordDetector(train=False)

In [5]:
train_loader = DataLoader(
    train_data, 
    batch_size=64,
    shuffle=True
)
val_loader = DataLoader(
    val_data, 
    batch_size=64,
    shuffle=True
)

In [6]:
class GRU(nn.Module):
    def __init__(self, input_size = 12, hidden_size = 128, num_layers = 1, num_classes = 12, bidirectional = True) -> None:
        super(GRU, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional
        
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first = True, bidirectional=bidirectional)
        if(bidirectional):
            self.fc = nn.Linear(hidden_size*2, num_classes)
        else:
            self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        if(self.bidirectional):
            h0 = torch.zeros(2*self.num_layers, x.size(0), self.hidden_size)
        else:
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.gru(x, h0)
        out = out[:,-1,:] # Since we only want the output of the last cell
        out = self.fc(out)
        return(out)

In [7]:
device = torch.device('cpu')
EPOCHS = 100

In [8]:
model = GRU().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 2e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, 0.995)

In [9]:
def predict(model, audio, chroma_req = True, chord_templates:dict = json.load(open('./chord_templates.json')), sr = 44100, hop = 256):
    if chroma_req:
        chroma = torch.Tensor(librosa.feature.chroma_cens(y=audio, sr = sr, hop_length=hop)).T.unsqueeze(0)
    else:
        chroma = audio
    with torch.no_grad():
        outputs = nn.functional.softmax(model(chroma), 1)[0]
    min_val = 120
    min_key = ''
    for key, val in chord_templates.items():
        out = torch.norm(torch.Tensor(val) - outputs)
        if min_val >= out:
            min_val = out
            min_key = key
    return min_key

In [16]:
best_accuracy = 0
best_weights = copy.deepcopy(model.state_dict())

In [17]:
for epoch in range(EPOCHS):
    model.train()
    for i, (chroma, labels) in enumerate(train_loader):
        chroma = chroma.to(device)
        labels = labels.to(device)

        outputs = model(chroma)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()
    with torch.no_grad():
        model.eval()
        count = 0
        for y in tqdm(val_data):
            pred = predict(model, y[0].unsqueeze(0), False)
            min_val = 120
            min_key = ''
            for key, val in chord_templates.items():
                out = torch.norm(torch.Tensor(val) - y[1])
                if min_val >= out:
                    min_val = out
                    min_key = key
            count+=(pred==min_key)
        acc = 100*count/len(val_data)
        if best_accuracy <= acc:
            best_accuracy = acc
            best_weights = copy.deepcopy(model.state_dict())
        print(f"Val Accuracy = {acc:.2f}%")
    print(f'Epoch {epoch+1}/{EPOCHS}; Loss: {loss.item():.4f}')

100%|██████████| 144/144 [00:05<00:00, 25.97it/s]


Val Accuracy = 11.81%
Epoch 1/100; Loss: 7.6251


100%|██████████| 144/144 [00:05<00:00, 26.00it/s]


Val Accuracy = 8.33%
Epoch 2/100; Loss: 7.6706


100%|██████████| 144/144 [00:05<00:00, 25.82it/s]


...

Val Accuracy = 94.44%
Epoch 98/100; Loss: 3.3063


100%|██████████| 144/144 [00:05<00:00, 25.78it/s]


Val Accuracy = 93.75%
Epoch 99/100; Loss: 3.3047


100%|██████████| 144/144 [00:05<00:00, 25.67it/s]

Val Accuracy = 94.44%
Epoch 100/100; Loss: 3.3221


In [18]:
model.load_state_dict(best_weights)
model.eval()

GRU(
  (gru): GRU(12, 128, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=12, bias=True)
)

In [19]:
y = librosa.load('./data/F#-Dim-1.wav')[0]
predict(model, y)

'F#dim'

In [20]:
y = librosa.load('./data/A#-Aug-1.wav')[0]
predict(model, y)

'F#aug'

In [13]:
count = 0
with torch.no_grad():
    for y in tqdm(val_data):
        model.eval()
        pred = predict(model, y[0].unsqueeze(0), False)
        min_val = 120
        min_key = ''
        for key, val in chord_templates.items():
            out = torch.norm(torch.Tensor(val) - y[1])
            if min_val >= out:
                min_val = out
                min_key = key
        count+=(pred==min_key)
print(f"Val Accuracy = {100*count/len(val_data):.2f}%")

100%|██████████| 144/144 [00:05<00:00, 27.34it/s]

Val Accuracy = 95.83%


In [24]:
torch.save(model.state_dict(), './models/chord_detector.pth')

In [10]:
model = GRU()
model.load_state_dict(torch.load('./models/chord_detector.pth'))
model.eval()

GRU(
  (gru): GRU(12, 128, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=12, bias=True)
)

In [18]:
y = librosa.load('./data/G-Minor-12.wav')[0]
predict(model, y)

'Gm'

In [12]:
count = 0
with torch.no_grad():
    for y in tqdm(train_data):
        model.eval()
        pred = predict(model, y[0].unsqueeze(0), False)
        min_val = 120
        min_key = ''
        for key, val in chord_templates.items():
            out = torch.norm(torch.Tensor(val) - y[1])
            if min_val >= out:
                min_val = out
                min_key = key
        count+=(pred==min_key)
print(f"Train Accuracy = {100*count/len(train_data):.2f}%")

100%|██████████| 576/576 [00:21<00:00, 27.37it/s]

Train Accuracy = 98.44%
